# هسته معنایی

در این نمونه کد، شما از چارچوب هوش مصنوعی [هسته معنایی](https://aka.ms/ai-agents-beginners/semantic-kernel) برای ایجاد یک عامل ساده استفاده خواهید کرد.

هدف این نمونه این است که مراحل مورد نیاز را به شما نشان دهد، مراحلی که در نمونه‌های کد بعدی هنگام پیاده‌سازی الگوهای مختلف عامل‌محور استفاده خواهیم کرد.


## وارد کردن بسته‌های مورد نیاز پایتون


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## ایجاد کلاینت

در این نمونه، ما از [مدل‌های GitHub](https://aka.ms/ai-agents-beginners/github-models) برای دسترسی به LLM استفاده خواهیم کرد.

`ai_model_id` به صورت `gpt-4o-mini` تعریف شده است. سعی کنید مدل را به مدل دیگری که در بازار مدل‌های GitHub موجود است تغییر دهید تا نتایج مختلف را مشاهده کنید.

برای استفاده از `Azure Inference SDK` که برای `base_url` مدل‌های GitHub استفاده می‌شود، ما از کانکتور `OpenAIChatCompletion` در Semantic Kernel استفاده خواهیم کرد. همچنین کانکتورهای دیگری نیز [در دسترس هستند](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) که می‌توان از Semantic Kernel برای ارائه‌دهندگان مدل‌های دیگر استفاده کرد.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ایجاد عامل

در اینجا عامل با نام `TravelAgent` را ایجاد می‌کنیم.

برای این مثال، از دستورالعمل‌های بسیار ساده‌ای استفاده می‌کنیم. می‌توانید این دستورالعمل‌ها را تغییر دهید تا ببینید عامل چگونه به شکل متفاوتی واکنش نشان می‌دهد.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## اجرای عامل

اکنون می‌توانیم عامل را با تعریف یک رشته از نوع `ChatHistoryAgentThread` اجرا کنیم. هر پیام سیستمی مورد نیاز به آرگومان کلیدواژه `messages` در `invoke_stream` عامل ارائه می‌شود.

پس از تعریف این موارد، یک `user_inputs` ایجاد می‌کنیم که همان چیزی است که کاربر به عامل ارسال می‌کند. در این مثال، این پیام را به `Plan me a sunny vacation` تنظیم کرده‌ایم.

می‌توانید این پیام را تغییر دهید تا ببینید عامل چگونه به شکل متفاوتی پاسخ می‌دهد.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
